In [2]:
import pandas as pd
import numpy as np
from datetime import date
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv(r'C:\Users\Dharmendra\Downloads\Receipts Data CSV (1).csv',encoding='cp1252')

In [3]:
data.dropna(inplace=True)
data

,CustomerId,CustomerName,CustomerType,CreditLimit,CreditDays,AverageOrderSize,LengthOfRelationship,InvoiceId,InvoiceDate,InvoiceAmount,PaymentDate,PaymentAmount,PaymentMode
0,OC601487,Rouat Al Suwaiq Intl LLc,TC,2500.0,0,11864.07,773.0,650511000042,2020-12-23,378.350,2023-01-04,11.350,CASH
1,OC211613,Janoob Al Madeena - Barka Gatemall,Credit,15000.0,CM60,41209.58,791.0,601501000212,2020-12-23,222.675,2023-01-22,119.137,CASH
2,OC176296,Mehmood Bin Saif - Barka,Credit,3000.0,CM30,10253.67,792.0,601501000244,2020-12-28,136.650,2023-01-22,98.150,CASH
3,OC603017,Al Mataeb Shopping,TC,5000.0,0,9337.49,778.0,640301000123,2021-01-10,408.025,2023-01-09,12.250,CASH
4,OC602808,Al Maqloodi Trd Co.LLC,TC,5000.0,0,12780.11,778.0,640301000139,2021-01-11,399.400,2023-01-09,13.975,CASH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146867,OC600091,Yeti Shopping,TC,5000.0,CM30,5572.18,862.0,601511002970,2023-04-03,8.875,2023-04-03,8.875,CASH
146868,OC601765,White LuLu Shoping,TC,800.0,0,673.79,862.0,650311005588,2023-04-03,9.850,2023-04-03,9.850,CASH
146869,OC602905,Kharan Muntrib,TC,5000.0,0,687.72,862.0,630211006111,2023-04-03,25.900,2023-04-03,8.150,CASH
146870,UC716015,Acura Mini Mart,TC,5000.0,0,4970.38,867.0,400411007884,2023-04-03,1416.250,2023-04-03,1416.250,CASH


dropping the nan values

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146872 entries, 0 to 146871
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   CustomerId            146872 non-null  object 
 1   CustomerName          146872 non-null  object 
 2   CustomerType          146872 non-null  object 
 3   CreditLimit           146872 non-null  float64
 4   CreditDays            146872 non-null  object 
 5   AverageOrderSize      146872 non-null  float64
 6   LengthOfRelationship  146872 non-null  float64
 7   InvoiceId             146872 non-null  object 
 8   InvoiceDate           146872 non-null  object 
 9   InvoiceAmount         146872 non-null  float64
 10  PaymentDate           146872 non-null  object 
 11  PaymentAmount         146872 non-null  float64
 12  PaymentMode           146872 non-null  object 
dtypes: float64(5), object(8)
memory usage: 15.7+ MB


In [5]:
data[['InvoiceDate']]=pd.DataFrame(pd.to_datetime(data['InvoiceDate']))

converting the invoicedata column to datetime

In [6]:
days_between_invoices = data.groupby('CustomerId')['InvoiceDate'].diff().dt.days

In [7]:
average_days_between_invoices = days_between_invoices.groupby(data['CustomerId']).mean()

In [8]:
average_days_between_invoices

CustomerId
96600001     8.000000
96600016    11.714286
96600020     5.789474
96600021     8.000000
96600022    14.500000
              ...    
UC717035    21.000000
UC717036    15.666667
UC717039    21.000000
UC717041    19.000000
UC717044    17.500000
Name: InvoiceDate, Length: 16049, dtype: float64

In [9]:
order_frequency = (average_days_between_invoices).fillna(0)

In [10]:
order_frequency

CustomerId
96600001     8.000000
96600016    11.714286
96600020     5.789474
96600021     8.000000
96600022    14.500000
              ...    
UC717035    21.000000
UC717036    15.666667
UC717039    21.000000
UC717041    19.000000
UC717044    17.500000
Name: InvoiceDate, Length: 16049, dtype: float64

In [11]:
of = pd.DataFrame(order_frequency).reset_index()
of_dict = of.to_dict()

In [12]:
of_dicts = dict(zip(of_dict['CustomerId'].values(),of_dict['InvoiceDate'].values()))

In [13]:
of_list = []
for i in range(len(data)):
    of_list.append(of_dicts.get(data[['CustomerId']].loc[i][0]))

In [14]:
len(of_list)

146872

In [15]:
data[['order_frequency']] = pd.DataFrame([of_list]).T

adding the order_frequency column to data

In [16]:
days_since_invoice = (pd.to_datetime(date.today()) - pd.to_datetime(data['InvoiceDate'])).dt.days

In [17]:
days_since_last_invoice = days_since_invoice.groupby(data['CustomerId']).min()

In [18]:
days_since_last_payment = (pd.to_datetime(date.today()) - pd.to_datetime(data['PaymentDate'])).dt.days.groupby(data['CustomerId']).min()

In [19]:
average_payment_amount = data.groupby('CustomerId')['PaymentAmount'].mean()

In [20]:
average_invoice_amount = data.groupby('CustomerId')['InvoiceAmount'].mean()

In [21]:
def f1(x):
    x = x.strip('CM')
    return x

In [22]:
data[['CreditDays']] = pd.DataFrame(data['CreditDays'].apply(f1).apply(int))

changing the creditdays columns to numerical column

In [23]:
average_days_late = (days_since_last_payment) - data['CreditDays'].groupby(data['CustomerId']).min()

In [24]:
late_payment_frequency = (days_since_last_payment) - data['CreditDays'].groupby(data['CustomerId']).count()

In [26]:
features = data.drop(['CustomerId', 'InvoiceDate','CreditLimit','CustomerName','InvoiceId','PaymentDate'], axis=1)
target = data[['CreditLimit']]

### SPLITING OF DATA TO TRAIN AND TEST

In [27]:
train_features = features[0:round(146872*0.8)]
train_target = target[0:round(146872*0.8)]
test_features = features[round(146872*0.8):]
test_target = target[round(146872*0.8):]

In [28]:
test_features_index = test_features.index

In [29]:
test_features.reset_index(drop=True,inplace=True)
test_target.reset_index(drop=True,inplace=True)

In [30]:
pipeline = Pipeline([
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [31]:
obj_columns = list(features.select_dtypes(include='object').columns)

### CATOGIRICAL TO NUMERICAL AND STANDARIZING THE COLUMNS 

In [32]:
ohe = OneHotEncoder()
a = ohe.fit_transform(train_features[obj_columns]).toarray()
ohe_data = pd.DataFrame(a,columns=['ct_CASH', 'Credit', 'TC', 'pm_CASH', 'CHEQUE', 'ONLINE', 'POS'])

In [33]:
train_features = train_features.join(ohe_data)

In [34]:
train_features.drop(obj_columns,axis=1,inplace=True)

In [35]:
std = StandardScaler()

In [36]:
train_features = pd.DataFrame(std.fit_transform(train_features))

In [37]:
b = ohe.transform(test_features[obj_columns]) .toarray()

In [38]:
ohe_test_data = pd.DataFrame(b,columns=['ct_CASH', 'Credit', 'TC', 'pm_CASH', 'CHEQUE', 'ONLINE', 'POS'])

In [39]:
ohe_test_data

,ct_CASH,Credit,TC,pm_CASH,CHEQUE,ONLINE,POS
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
29369,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29370,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29371,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29372,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [40]:
test_features = test_features.join(ohe_test_data)

In [41]:
test_features

,CustomerType,CreditDays,AverageOrderSize,LengthOfRelationship,InvoiceAmount,PaymentAmount,PaymentMode,order_frequency,ct_CASH,Credit,TC,pm_CASH,CHEQUE,ONLINE,POS
0,TC,30,35647.47,730.0,1021.650,1021.650,CASH,7.375000,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,TC,30,16300.90,710.0,2923.550,500.000,CASH,7.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,TC,30,40285.64,722.0,2211.090,150.000,CASH,3.758621,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,TC,30,25648.04,703.0,69.410,69.410,CASH,4.388889,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,TC,30,13555.68,705.0,2555.440,2555.440,POS,5.923077,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29369,TC,30,5572.18,862.0,8.875,8.875,CASH,5.740741,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29370,TC,0,673.79,862.0,9.850,9.850,CASH,19.500000,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29371,TC,0,687.72,862.0,25.900,8.150,CASH,7.875000,0.0,0.0,1.0,1.0,0.0,0.0,0.0
29372,TC,0,4970.38,867.0,1416.250,1416.250,CASH,12.800000,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [42]:
test_features.drop(obj_columns,axis=1,inplace=True)

In [43]:
test_features = pd.DataFrame(std.transform(test_features))

### Training the model

In [44]:
pipeline.fit(train_features, train_target)

C:\Users\Dharmendra\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('regressor', RandomForestRegressor(random_state=42))])

### Predicting the test data (Evaluating)

In [45]:
predicted_credit_scores = pipeline.predict(test_features)

In [46]:
suggested_credit_limits = pd.Series(predicted_credit_scores, index=test_features.index)

In [54]:
suggested_credit_limits_changed = suggested_credit_limits.apply(lambda x: x*1.1 if x < 8000 else x)

In [55]:
suggested_credit_limits_changed

0        9668.000
1        3235.375
2        3235.375
3        5226.265
4        4633.805
           ...   
29369    4766.630
29370    4326.300
29371    3967.700
29372    6820.000
29373    7062.000
Length: 29374, dtype: float64

In [56]:
data1 = data

In [57]:
data1[['SuggestedCreditLimit']]=0

adding the SuggestedCreditLimit to dataframe with dummy value 0

In [63]:
data1.loc[test_features_index, 'SuggestedCreditLimit'] = suggested_credit_limits_changed.values

### Dataframe to CSV 

In [69]:
data1.to_csv(r'C:\Users\Dharmendra\Downloads\vamsidhar_output.csv',index=False)